In our second model, and after doing some research, we tried to leave the numerical data in the model instead of dropping it in order to avoid data loss in hopes the model would improve.   While the model did see significant increase in the accuracy to 51.8% with a loss of 1.23, it still fell well short of the desired threshold to consider the model successful.

In [1]:
#Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
#Import and read the dataset.
dataset_df = pd.read_csv("data/housing.csv")
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
#Identify incomplete rows
dataset_df.count()

longitude             20640
latitude              20640
housing_median_age    20640
total_rooms           20640
total_bedrooms        20433
population            20640
households            20640
median_income         20640
median_house_value    20640
ocean_proximity       20640
dtype: int64

In [4]:
#Drop all rows with missing information
dataset_df = dataset_df.dropna(how="any")
dataset_df.count()

longitude             20433
latitude              20433
housing_median_age    20433
total_rooms           20433
total_bedrooms        20433
population            20433
households            20433
median_income         20433
median_house_value    20433
ocean_proximity       20433
dtype: int64

In [5]:
#Find the minimum median_house_value for binning.
dataset_df["median_house_value"].min()

14999.0

In [6]:
#Find the maximum median_house_value for binning.
dataset_df["median_house_value"].max()

500001.0

In [8]:
#Bin the median_house_value.

#Create bin cutoffs
value_bins = [14999, 68888, 122777, 176666, 230555, 284444, 338333, 392222, 446111, 500001]

#Create bin names
value_names = ["14999-68888", "68889-122777", "122778-176666", "176667-230555", "230556-284444", "284445-338333", "338334-392222", "392223-446111", "446112-500001"]

#Slice the data and place it into bins
dataset_df["value_category"] = pd.cut(dataset_df["median_house_value"], value_bins, labels=value_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,value_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,446112-500001
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,338334-392222
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,338334-392222
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,338334-392222
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,338334-392222


In [9]:
#Drop original columns that were binned.
dataset_df = dataset_df.drop(columns=["median_house_value"])
dataset_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,value_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,446112-500001
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,338334-392222
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,338334-392222
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,338334-392222
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,338334-392222
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND,68889-122777
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND,68889-122777
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND,68889-122777
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND,68889-122777


In [10]:
home_values_df = pd.DataFrame(dataset_df['value_category'])
home_values_encoded_df = pd.get_dummies(home_values_df)
home_values_encoded_df.head(5)

,value_category_14999-68888,value_category_68889-122777,value_category_122778-176666,value_category_176667-230555,value_category_230556-284444,value_category_284445-338333,value_category_338334-392222,value_category_392223-446111,value_category_446112-500001
0,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,True,False,False


In [11]:
convert_boolean = home_values_encoded_df.select_dtypes(include=['bool']).columns
home_values_encoded_df[convert_boolean] = home_values_encoded_df[convert_boolean].astype(int)

home_values_encoded_df.head(5)

,value_category_14999-68888,value_category_68889-122777,value_category_122778-176666,value_category_176667-230555,value_category_230556-284444,value_category_284445-338333,value_category_338334-392222,value_category_392223-446111,value_category_446112-500001
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0


In [12]:
new_dataset_df = dataset_df.drop(columns=['value_category'])

new_dataset_df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY


In [13]:
conv_dataset_df = pd.get_dummies(new_dataset_df, columns=['ocean_proximity'])
conv_dataset_df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,False,False,False,True,False
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,False,False,False,True,False
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,False,False,False,True,False
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,False,False,False,True,False
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,False,False,False,True,False


In [14]:
conv_dataset_df = conv_dataset_df * 1
conv_dataset_df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0,0,0,1,0


In [15]:
#Split processed data into features and target arrays
y = home_values_encoded_df.values
x = conv_dataset_df.values

#Split the processed data into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)


In [16]:
#Create a StandardScaler instance
scaler = StandardScaler()

#Fit the StandardScaler
x_scaler = scaler.fit(x_train)

#Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [34]:
nn = tf.keras.models.Sequential()

input_features = x.shape[1]

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu',input_dim=input_features))

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=75, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=9, activation='softmax'))

# Check model structure
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 40)                560       
                                                                 
 dense_18 (Dense)            (None, 9)                 369       
                                                                 
Total params: 929 (3.63 KB)
Trainable params: 929 (3.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# Compile the model
nn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
# Train the model
train_model = nn.fit(x_train_scaled,y_train,epochs=100)

Epoch 1/100
479/479 [==============================] - 1s 1ms/step - loss: 1.7040 - accuracy: 0.3667
Epoch 2/100
479/479 [==============================] - 1s 1ms/step - loss: 1.4601 - accuracy: 0.4355
Epoch 3/100
479/479 [==============================] - 1s 1ms/step - loss: 1.4003 - accuracy: 0.4546
Epoch 4/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3709 - accuracy: 0.4620
Epoch 5/100
479/479 [==============================] - 1s 2ms/step - loss: 1.3527 - accuracy: 0.4685
Epoch 6/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3401 - accuracy: 0.4730
Epoch 7/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3296 - accuracy: 0.4763
Epoch 8/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3201 - accuracy: 0.4792
Epoch 9/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3132 - accuracy: 0.4814
Epoch 10/100
479/479 [==============================] - 1s 1ms/step - loss: 1.3064 - accura

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

160/160 - 0s - loss: 1.2305 - accuracy: 0.5187 - 432ms/epoch - 3ms/step
Loss: 1.2305495738983154, Accuracy: 0.5186924934387207
